#### Ask GPT for answers for training

In [123]:
import openai
import os
import tqdm


In [124]:
#/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/formatted_scores.json
#/Users/seanhuang/Grad/
formatted_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/formatted_scores.json"
receipt_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/receipt_scores.json"
report_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/report_scores.json"
random_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/random_scores.json"

In [125]:
import json
#load and sort the data inferenced by on device model
with open(formatted_pdfs, "r") as f1, open(receipt_pdfs, "r") as f2, open(report_pdfs, "r") as f3, open(random_pdfs, "r") as f4:
    formatted = json.load(f1)
    receipts = json.load(f2)
    reports = json.load(f3)
    randoms = json.load(f4)
    print("PDFs with formats",len(formatted))
    print("PDFs of recipts that need ocr", len(receipts))
    print("PDFs of well written long papers, no ocr",len(reports))
    print("PDFs of all sorts",len(randoms))
formatted = sorted(formatted, key=lambda x: list(x.keys())[0])
receipts = sorted(receipts, key=lambda x: list(x.keys())[0])
reports = sorted(reports, key=lambda x: list(x.keys())[0])
randoms = sorted(randoms, key=lambda x: list(x.keys())[0])
print (formatted[0])
print (receipts[0])

PDFs with formats 4840
PDFs of recipts that need ocr 2441
PDFs of well written long papers, no ocr 98
PDFs of all sorts 600
{'/home/yxuhuang/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_1_normal.txt': {'Scores': '8', 'Chunks': 'Stock Report for 2016-07 \n Category : Beverages  \n id category : 1  \nProduct\nUnits Sold\nUnits in Stock\nUnit Price\nChang\n105\n17\n19\nGuaraná Fantástica\n43\n20\n4.5\nSteeleye Stout\n20\n20\n18\nChartreuse verte\n48\n69\n18\nOutback Lager\n41\n15\n15\nLakkalikööri\n15\n57\n18\n\n--------------------------------------------------\n', 'Summaries': 'Stock Report for 2016-07 \n id category : 1 \nBeverages: \n- Guaraná Fantástica: 43 units sold, 20 in stock, $4.5/unit \n- Steeleye Stout: 20 units sold, 20 in stock, $18/unit \n- Chartreuse verte: 48 units sold, 69 in stock, $18/unit \n- Outback Lager: 41 units sold, 15 in stock, $15/unit \n- Lakkalikööri: 15 u

#### Training will happen after the first summary for both ocr detection and text extraction versions. 
##### The input now is no longer gibberish and not natural language.  
##### Now we need to suppress the apologies/commentaries on sensitive data and gibberish text. There is no need to train on the initial summary process since it will skew the model into detecting random gibberish.

In [126]:
#convert the scores to average and omit the scores when summaries are not usable
def get_average_scores(pdf, separator : str, filtered_txt = [""]):
    #print(filtered_txt)
    file_name = list(pdf.keys())[0]
    values = list(pdf.values())[0]
    scoretxt =  values.get("Scores")
    scoretxt = scoretxt.strip()
    scores = scoretxt.split(separator)
    score = 0.0
    count = 0
    summaries = values.get("Summaries")
    summaries = summaries.split(separator)
    for s, txt in zip(scores, summaries): 
        lower_txt = txt.lower()
        #print(s , txt)
        for filt in filtered_txt:
            if filt.lower() in lower_txt:
                #print(f"In file {file_name} filtered", txt)
                break
            else:
                try:
                    score += float(s)
                    count += 1
                except ValueError:
                    continue
    return 0 if count == 0 else score/count

def count_pdf_available_and_filtered_chunks(pdf, separator : str, filtered_txt = [""]):
    file_name = list(pdf.keys())[0]
    available_chunks = []
    filtered_chunks = []
    values = list(pdf.values())[0]
    chunks = values.get("Chunks").split(separator)
    summaries = values.get("Summaries").split(separator)
    for c ,summary  in zip(chunks,summaries): 
        lower_txt = summary.lower()
        if any(filt.lower() in lower_txt for filt in filtered_txt):
            #print(f"In file {file_name} filtered", txt)
            filtered_chunks.append(c)
        else:
            available_chunks.append(c)
    #print("ava vs filt:",len(available_chunks), len(filtered_chunks))
    return available_chunks, filtered_chunks
        


In [127]:
def get_higher_scored_pdf(pdfs, separator : str, filtered_txt = [""]):
    if len(pdfs) == 0:
        return None
    solution = None
    score = 0 
    for pdf in pdfs:
        if solution == None:
            solution = pdf
            score = get_average_scores(pdf, separator,filtered_txt=filtered_txt)
        else:
            new_score = get_average_scores(pdf, separator, filtered_txt=filtered_txt)
            if new_score > score:
                solution = pdf
                score = new_score

    return solution

In [128]:
import re
## get sorted adjacent pdfs that originated from the same source
def get_adjacent_pdfs(pdfs, start, reg = r'_(normal|ocr)\.txt$'):
    adjacent = []
    i = start
    while i < len(pdfs):
        if i == start:
            adjacent.append(pdfs[i])
        else:
            prev = re.sub(reg , '', list(adjacent[-1].keys())[0])
            current = re.sub(reg , '', list(pdfs[i].keys())[0])
            if prev == current:
                adjacent.append(pdfs[i])
            else:
                break
        i += 1
    return adjacent, i

In [139]:
def get_the_best_version(pdfs, separator: str, filtered_txt):
    higher_scored_pdfs = []
    higher_scored_filtered = []
    i = 0
    useful_chunks = []
    filtered_chunks = []
    while i < len(pdfs):
        start = i
        adjacents, i = get_adjacent_pdfs(pdfs, start)
        # for f in adjacents:
        #     print(f"PDF from: {start} to {i}:",f)
        winner = get_higher_scored_pdf(adjacents, separator=separator, filtered_txt=filtered_txt)
        useful, filtered = count_pdf_available_and_filtered_chunks(winner, separator=separator, filtered_txt=filtered_txt) 
        if len(filtered) != 0:
            higher_scored_filtered.append(winner)
        higher_scored_pdfs.append(winner)
        #print("Incoming:", len(useful), len(filtered))
        useful_chunks.extend(useful)
        filtered_chunks.extend(filtered)
        #print("Current Lenghts:",len(useful_chunks), len(filtered_chunks))
        
        #print("Winner", winner)
    return higher_scored_pdfs, higher_scored_filtered, useful_chunks, filtered_chunks 
def get_censored_text(text):
    return text.replace("CENSORED", " ").replace("censored", " ")


##### The resulted data will still contain appologies and commentaries but it should contain the best availale information from the original pdf.

In [140]:
keywords = [ 
        "I can't",
        "I cannot",
        "There is no",
        "No useful",
        "Unusable"]
#formatted inputs contains pdfs with a lot of tables, and forms, most of them are text based
formatted_inputs ,filtered_formatted_inputs , useful_formatted_chunks, filtered_formatted_chunks = get_the_best_version(formatted, separator="\n}<->{\n", filtered_txt= keywords)

#receipt inputs contains pdfs that are mostly images with text, and need ocr 
receipt_inputs, filtered_receipt_inputs, useful_receipt_chunks, filtered_receipt_chunks =  get_the_best_version(receipts, separator="\n}<->{\n", filtered_txt= keywords)

#report inputs contains pdfs that are well written, and no ocr is needed
report_inputs ,filtered_report_inputs , useful_report_chunks, filtered_report_chunks = get_the_best_version(reports, separator="\n}<->{\n", filtered_txt= keywords)

#random inputs contains pdfs that are all sorts mix of ocr and text
random_inputs, filetered_random_inputs, useful_random_chunks, filtered_random_chunks = get_the_best_version(randoms, separator="\n}<->{\n", filtered_txt= keywords)

print(f"PDFs with formats {len(formatted_inputs)}, that are filtered {len(filtered_formatted_inputs)}, useful chunks {len(useful_formatted_chunks)}, filtered chunks {len(filtered_formatted_chunks)}")
print(f"PDFs of recipts that need ocr {len(receipt_inputs)}, that are filtered {len(filtered_receipt_inputs)}, useful chunks {len(useful_receipt_chunks)}, filtered chunks {len(filtered_receipt_chunks)}")
print(f"PDFs of well written long papers, no ocr {len(report_inputs)}, that are filtered {len(filtered_report_inputs)}, useful chunks {len(useful_report_chunks)}, filtered chunks {len(filtered_report_chunks)}")
print(f"PDFs of all sorts {len(random_inputs)}, that are filtered {len(filetered_random_inputs)}, useful chunks {len(useful_random_chunks)}, filtered chunks {len(filtered_random_chunks)}")




for i in formatted_inputs[0:5]:
    print(i)
for i in receipt_inputs[0:5]:
        print(i)
for i in report_inputs[0:5]:
        print(i)
for i in random_inputs[0:5]:
        print(i)

PDFs with formats 2599, that are filtered 366, useful chunks 4340, filtered chunks 366
PDFs of recipts that need ocr 1222, that are filtered 709, useful chunks 3807, filtered chunks 1204
PDFs of well written long papers, no ocr 98, that are filtered 21, useful chunks 14371, filtered chunks 30
PDFs of all sorts 508, that are filtered 188, useful chunks 24405, filtered chunks 694
{'/home/yxuhuang/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_1_normal.txt': {'Scores': '8', 'Chunks': 'Stock Report for 2016-07 \n Category : Beverages  \n id category : 1  \nProduct\nUnits Sold\nUnits in Stock\nUnit Price\nChang\n105\n17\n19\nGuaraná Fantástica\n43\n20\n4.5\nSteeleye Stout\n20\n20\n18\nChartreuse verte\n48\n69\n18\nOutback Lager\n41\n15\n15\nLakkalikööri\n15\n57\n18\n\n--------------------------------------------------\n', 'Summaries': 'Stock Report for 2016-07 \n id category : 1 \nBeverages:

#### There are 4 types of data:
* Formatted data has a lot of numbers, we could make the model to adhere to addressing the numbers formally or concisely.
* Receipts are mainly ocred and contains  a lot of personal which produced highest rate of disclamers and commentaries so commentary suppression data can be gathered on it.
* Papers contains well written text and information dense, we could train the model to be more informative and concise.
* Mixed PDFs' content are a mix of the above types, we could try to mimic the tone of the original text.


In [229]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI
from openai._exceptions import RateLimitError, APITimeoutError, APIError
import json
import time
from pathlib import Path

# Load .env and initialize OpenAI client
load_dotenv()
api_key = os.getenv("GPT_API_KEY")
gpt_client = OpenAI(api_key=api_key)
def summarize_chunk(text: str, prompt, model, retry=True) -> str:
    prompt = f"""{prompt}
Text:
{text}
"""
    try:
        #print("Sending:",prompt)
        response = gpt_client.chat.completions.create(
            model= model,
            messages=[
                {"role": "system", "content": "You are a multi-lingo helpful factual text-edit assitant."},
                {"role": "user", "content": prompt}],
            temperature=0.2,
        )
        #print("Response:",response) 
        # Check structure before returning
        if response and response.choices and response.choices[0].message:
            return response.choices[0].message.content.strip()
        else:
            print("Warning: Unexpected response format.")
            return ""

    except RateLimitError:
        print("Rate limit hit. Sleeping for 10 seconds...")
        if retry:
            time.sleep(10)
            return summarize_chunk(text, prompt,model=model)  # retry

    except APITimeoutError:
        print("Request timed out. Retrying...")
        if retry:
            return summarize_chunk(text, prompt,model=model)

    except APIError as e:
        print(f"OpenAI API error: {e}")
        return "<Error>"
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "<Error>"


def process_chunks_and_save( data_list, model, output_dir, file_name , summary_type, prompt, retry=True):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for i, chunk in enumerate(data_list):
        chunk = chunk.strip()
        if not chunk:
            continue

        try:
            summary = summarize_chunk(chunk, prompt,model=model, retry=retry)
            if summary:
                result = {
                    "input": chunk,
                    "output": summary,
                    "summary_type": summary_type, 
                }
                with open(output_path / file_name, "a", encoding="utf-8") as f:
                    json.dump(result, f, ensure_ascii=False, indent=2)
            else:
                print(f"Suppressed")
        except Exception as e:
            print(f"[{i+1}] Error: {e}")
def process_filtered_chunks_and_save( data_list, output_dir, file_name , summary_type="Filtered"):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for i, chunk in enumerate(data_list):
        chunk = chunk.strip()
        if not chunk:
            continue
        result = {
            "input": chunk,
            "output": "<NULL>",
            "summary_type": summary_type, 
        }
        with open(output_path / file_name, "a", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
         

##### Pick random useful chunks should only be executed once for file creation

In [ ]:
import random
import json
sample_size = 1000
chosen_formatted = random.sample(useful_formatted_chunks, sample_size)
chosen_receipts = random.sample(useful_receipt_chunks, sample_size)
chosen_reports = random.sample(useful_report_chunks, sample_size)
chosen_randoms = random.sample(useful_random_chunks, sample_size)
#with open('../Training/Input/chosen_formatted.json', 'w') as f:
    #json.dump(chosen_formatted, f, indent=2)

#with open('../Training/Input/chosen_receipts.json', 'w') as f:
    #json.dump(chosen_receipts, f, indent=2)

#with open('../Training/Input/chosen_reports.json', 'w') as f:
    #json.dump(chosen_reports, f, indent=2)

#with open('../Training/Input/chosen_randoms.json', 'w') as f:
    #json.dump(chosen_randoms, f, indent=2)

filtered_sample_size = 100
#contains mostly lines, so not difficult to summarize
chosen_filtered_formatted = random.sample(filtered_formatted_chunks, min(filtered_sample_size, len(filtered_formatted_chunks)))
chosen_filtered_receipts = random.sample(filtered_receipt_chunks, min(filtered_sample_size, len(filtered_receipt_chunks)))
chosen_filtered_reports = random.sample(filtered_report_chunks, min(filtered_sample_size, len(filtered_report_chunks)))
chosen_filtered_randoms = random.sample(filtered_random_chunks, min(filtered_sample_size, len(filtered_random_chunks)))
with open('../Training/Input/chosen_filtered_formatted.json', 'w') as f:
    json.dump(chosen_filtered_formatted, f, indent=2)
with open('../Training/Input/chosen_filtered_receipts.json', 'w') as f:
    json.dump(chosen_filtered_receipts, f, indent=2)
with open('../Training/Input/chosen_filtered_reports.json', 'w') as f:
    json.dump(chosen_filtered_reports, f, indent=2)
with open('../Training/Input/chosen_filtered_randoms.json', 'w') as f:
    json.dump(chosen_filtered_randoms, f, indent=2)


In [233]:
process_filtered_chunks_and_save( chosen_filtered_formatted, "../Training/InputNLabel/filtered_labels", "filtered_formatted.json", "Formatted")
process_filtered_chunks_and_save( chosen_filtered_receipts, "../Training/InputNLabel/filtered_labels", "filtered_receipts.json", "Receipt")
process_filtered_chunks_and_save( chosen_filtered_reports, "../Training/InputNLabel/filtered_labels", "filtered_reports.json", "Report")
process_filtered_chunks_and_save( chosen_filtered_randoms, "../Training/InputNLabel/filtered_labels", "filtered_randoms.json", "Random")

In [ ]:
#print(len(useful_formatted_chunks[0]))
import random
chosen_example = chosen_receipts[250]
process_chunks_and_save([chosen_example], model="gpt-4" , retry=True, output_dir="../Training", file_name="formatted_test.json", 
                        summary_type="formatted_without_bias", prompt= """Without commentary, summarize in original languange on useful information, below 100 If no meaning return <NULL>, if the content includes sensitive personal data, return <CENSORED>.""")
print(len(random.sample(useful_random_chunks, 2)))

2


In [ ]:
chosen_example = chosen_receipts[801]
process_chunks_and_save([chosen_example], model="gpt-4" , retry=True, output_dir="../Training", file_name="formatted_test.json", 
                        summary_type="formatted_without_bias", prompt= """Without commentary, summarize in original languange on useful information, below 100 words. If no meaning return <NULL>, if the content includes sensitive personal data, return <CENSORED>.""")
print(len(random.sample(useful_random_chunks, 2)))

2


In [ ]:
chosen_example = chosen_receipts[801]
process_chunks_and_save([chosen_example], model="gpt-4" , retry=True, output_dir="../Training", file_name="formatted_test.json", 
                        summary_type="sensitive_included_from original", prompt= """Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>.""")


In [ ]:
##Taking each's first 300
#block size 2: changed prompt
block_size_2 = 300
startPage = 2
for i in range(block_size_2 * startPage, block_size_2 * (startPage + 1), block_size_2):
    begin = i
    end = min(i + block_size_2, sample_size)

    process_chunks_and_save(
        chosen_receipts[begin:end],
        model="gpt-4",
        retry=True,
        output_dir="../Training/InputNLabel/prompt_sensitive_translated/",
        file_name=f"receipts_{i//block_size_2}.json",
        summary_type="receipts",
        prompt= """Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>."""
    )
    process_chunks_and_save(
        chosen_formatted[begin:end],
        model="gpt-4",
        retry=True,
        output_dir="../Training/InputNLabel/prompt_sensitive_translated/",
        file_name=f"formatted_{i//block_size_2}.json",
        summary_type="formatted",
        prompt="""Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>."""
    )


    process_chunks_and_save(
        chosen_reports[begin:end],
        model="gpt-4",
        retry=True,
        output_dir="../Training/InputNLabel/prompt_sensitive_translated/",
        file_name=f"reports_{i//block_size_2}.json",
        summary_type="reports",
        prompt="""Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>."""
    )

    process_chunks_and_save(
        chosen_randoms[begin:end],
        model="gpt-4",
        retry=True,
        output_dir="../Training/InputNLabel/prompt_sensitive_translated/",
        file_name=f"random_{i//block_size_2}.json",
        summary_type="random",
        prompt="""Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>."""
        )